# Generate a logo from a summary with DallE
- File output resolution are depending from the model selection
 - for dalle-e-3 the resolution maximum for dall-e-2 is 1024x1024
 - for dall-e-3 there are some new params like quality ( "standard" or "hd" ) and style ("natural" or "vivid"), the minimum size is 1024x1024

In [ ]:
%pip install openai --upgrade
%pip install dotenv #for local testing

In [1]:
import os, dotenv
from dotenv import load_dotenv # to load .env file
import openai, datetime, json, requests,os
from openai import OpenAI
from PIL import Image

In [3]:
openai.dalle_model = "dall-e-3" #you could choose "dall-e-3" or "dall-e-2

# For local
# Import Key and Base from .env
#load_dotenv("secrets.env") 
#openai.api_key = os.getenv("OPENAI_API_KEY")
#openai.organization = os.getenv("OPENAI_ORGANIZATION")


# param for have result more deterministic ( 0) or creative (1) you could have step for experimentations like 0.5 
temperature = 0

#for google Colab use
from google.colab import userdata, files
openai.api_key = userdata.get('OPENAI_API_KEY')
#openai.organization = userdata.get('OPENAI_ORGANIZATION')
# Verify if loading is correct
# print("Your OpenAI organization : " + openai.organization)


## Directory to save the generated images

In [6]:
#local 
#images_dir = "images"
#os.makedirs(images_dir, exist_ok=True)


# google colab files
!mkdir images
images_dir = "content/images"

In [7]:
# We create a client instance for openAI
client = OpenAI(
    api_key=openai.api_key,
    #organization=openai.organization
)

## Your idea for this logo

In [8]:
# Modify the text below to generate your own logo
yourIdea = "a horse with a shield and a sword with the named defHorsey"

#params for DallE
dalleStyle = "natural" # or vivid only for dalle-3
dalleQuality = "hd" # or standard only for dalle-3
dalleSize = "1024x1024" # for dalle-2, you could choose 256x256, 512x512, 1024x1024 / for Dalle-3,  1024x1024, 1792x1024, 1024x1792


In [9]:
#the prompt system in order to create a logo from a summary

promptSystem  = """You are a prompt engineer specialized in logo generation. You are working for a company who wants a new logo from a summary and a idea
you have to be creative in order to generate a logo that is unique and that will be used by the company for their branding. you could give a detailled description in order to use a text to image model
you will use this idea to generate a logo in"""

In [10]:
model =  "gpt-4"  #you could choose gpt-4 /gpt-4-32k / gpt-3.5-turbo / gpt-3.5-turbo-16k

response = client.chat.completions.create(
    model=model,
    n=1,
    temperature=temperature, # 0 to 1 by  step of 0.1 - O for  deterministic result, 1 is very creative
    messages=[
            {"role": "system", "content": promptSystem},
            {"role": "user", "content": yourIdea},
        ],
    )
    
promptImage = response.choices[0].message.content

print(promptImage)

Logo Description:

The logo will be a circular emblem, giving it a classic and timeless feel. The main element of the logo will be a majestic horse, standing tall and proud in the center. The horse will be stylized, not too detailed but enough to clearly identify it as a horse. It will be in a dynamic pose, as if it's ready to charge into battle, symbolizing strength and determination.

The horse will be in a rich, dark brown color, with lighter shades to highlight its muscles and features, giving it a 3D effect. The horse's mane and tail will be flowing, adding a sense of motion to the logo.

To the left of the horse, there will be a shield. The shield will be in a traditional shape, slightly curved and pointed at the bottom. It will be in a metallic silver color, with a few scratches and dents to show it's been in battle, symbolizing protection and resilience.

To the right of the horse, there will be a sword. The sword will be upright, with the blade pointing upwards. It will also b

### Generating image from the prompt

In [11]:
#we will use the prompt image from the prompt engineer to generate the logo
 
result = client.images.generate(
        prompt=promptImage,
        size=dalleSize,
        n=1,
        model=openai.dalle_model,
        style=dalleStyle,
        quality=dalleQuality,
)

results = json.loads(result.model_dump_json())
# Getting the url image (only the first one because n=1 )

dalleimage = requests.get(results["data"][0]["url"]).content

# Saving the image as a local file

dt = str(datetime.datetime.today().strftime("%d%b%Y_%H%M%S").lower())
image_file = os.path.join(
        images_dir, yourIdea[:50].replace(",", " ") + "_" + dt + ".png"
    )
with open(image_file, "wb") as imagefile:
    imagefile.write(dalleimage)

print(f"\nImage saved: {image_file}")

# Return the image as a PIL image file
img = Image.open(image_file)

img.show()

images\a horse with a shield and a sword with the named d_19nov2023_165201.png

Image saved: images\a horse with a shield and a sword with the named d_19nov2023_165201.png
